## Note that the .py version is what is actually imported. Only use this for testing.

In [2]:
# import statements
import numpy as np
import os
import torch
import uproot
from torch.utils.data import Dataset

In [3]:
class ProductionModeDataset(Dataset):
    """
    This class will load the minitree and create the dataset loaded into the nns
    
    0) load the minitree properly
    
    1) remove 0 and 2 type events so we have the same number as qqbar
    
    2) normalize all data
    
    3) ensure this is stored in a numpy array with the correct arrangement of data and a corresponding list of datatypes
    
    4) split this up into training and evaluating datasets
    
    5) provide methods with which these sets can be accessed
    
    Args:
        :param root (string): is root directory and file name of minitree
        :param split (boolean): tells whether to split into training and eval
        :param normalize (boolean): tells whether to normalize data
        :param remove (boolean): tells whether we should remove excess data for non-qqbar events or duplicate qqbar
        :param train (boolean): tells whether we are training or evaluating (probably not needed)
        :param correlation_cut(float): if positive, removes values correlated more than the correlation cut. Requires existing 
                                analysis of the given cut so that data can be loaded
        : param cut_version (int): if positive, loads the specific cut version (otherwise loads unnumbere OG version)
    """
    
    def __init__(self, root, split=True, normalize=True, remove=True, train=True, correlation_cut=-1.0, cut_version=-1):
        # load a correlation cut if it exists:
        to_remove = np.array(())  # initialize the array of inputs to remove because of cut as empty
        if correlation_cut > 0:
            # load the inputs which must be removed due to cut if positive
            if cut_version > 0:
                to_remove = np.load("../analysis_code/results/inputs_to_remove_cut_" + str(correlation_cut)\
                                    + "v" + str(cut_version) + ".npy")
                print("loaded correlations... shape is " + str(to_remove.shape))
            else:
                to_remove = np.load("../analysis_code/results/inputs_to_remove_cut_" + str(correlation_cut) + ".npy")
                print("loaded correlations... shape is " + str(to_remove.shape))
        
        self.events = uproot.open(root)   # open the root file and load the events to be processed
        self.training = train    # set a variable which determines whether this is loaded for training. probably not needed
        
        key = self.events.keys()[0]   # get the key to make accessing events easier
        data_list = self.events[key].keys()   # get the names of the inputs
        
        # make a numpy array from events array
        self.events_array = np.array([self.events[key + "/" + k].array(library="np") for k in data_list])
        print(data_list)
    
        self.events_array = np.transpose(self.events_array)   # turn into columns (instead of rows) of data

        
        ################# remove any inputs in to_remove from the events array and the list of inputs ################
        if to_remove.shape[0] > 0:
            self.events_array = np.delete(self.events_array, to_remove, 1)
            to_remove = np.sort(to_remove)
            for i in range(len(to_remove)):
                to_remove[i] -= i
                data_list.pop(to_remove[i])
        ##############################################################################################################
        
        
        if remove:
            """
            Here is where we remove the excess data so that qqbar, gg, and gq/qg events are equally
            represented in the training and analysis.
            """
            # find index of production_mode:
            index = 0
            for i in range(len(data_list)):
                if data_list[i] == "production_mode":
                    index = i
                    break
                    
            # sort events array by production mode
            self.events_array = self.events_array[np.argsort(self.events_array[:, index])]           
            
            ############## find first and last index with production mode 1 #########################
            first = 0  # this will be the first index with qqbar
            last = 0   # this will be the last index with qqbar
            found_first = False   # a flag which allows us to stop looking for first once found
            
            for i in range(len(self.events_array[:,index])):
                if not found_first:
                    if self.events_array[i ,index] == 1:
                        first = i
                        found_first = True
                
                if self.events_array[i,index] == 2:
                    last = i - 1
                    break
            #######################################################################################
                    
            num_qqbar = last + 1 - first  # this is the total number of qqbar events
            print("num qqbar = " + str(num_qqbar))
            
            ####################### remove the extra gg and other #############################################
            max_len = len(self.events_array[:,0])
            self.events_array = np.delete(self.events_array, list(range(num_qqbar, first)) + \
                                          list(range(max_len - (max_len - (last + 1) -num_qqbar), max_len)), 0)
            ###################################################################################################
            
  
        if normalize:
            """
            Here we normalize all the data by subtracting the minimum and dividing by the range (max-min). We do not normalize
            anything not going to be used as an input to the network (last three columns) except the weights which are only
            divided by the max to ensure we keep the signs.
            """
            # loop through all the inputs. the last three are not looped through because they are not input to the 
            # neural network. Hence, we subtract 3 from the length.
            for i in range(len(self.events_array[0,:])-3):
                ori_a = self.events_array[:,i] # original input column 

                min_a = np.min(self.events_array[:,i]) # min
                max_a = np.max(self.events_array[:,i]) # max
                range_a = max_a - min_a #range

                self.events_array[:, i] = (ori_a - min_a) / (range_a) # replace the og array with the normalized array
                
            #################### normalize the weights here: ######################################################
            n = len(self.events_array[0,:])-2   # this is the index of the weights for reco level inputs
            self.events_array[:,n] /= np.max(self.events_array[:,n])   # replace weights with normalized weights
            #######################################################################################################
        
        
               
        if split:
            """
            Here we split the events array into a validation dataset and a training dataset. 80% will be training and 
            20% will be validation.
            """
            # shuffle so no longer sorted by production modebefore splitting
            np.random.shuffle(self.events_array)
            
            train_size = int(len(self.events_array[:,0])*80/100)  # calculate the size of the training array
            
            # split into the two arrays
            self.train_array, self.eval_array = np.split(self.events_array, [train_size])
            print("training " + str(self.train_array.shape))
            print("evaluating " + str(self.eval_array.shape))
            
        else:
            """
            If not splitting, just set both the train_array and eval_array to be the entire dataset
            """
            self.train_array = self.events_array
            self.eval_array = self.events_array
                
    def __getitem__(self, index):
        """
        Return the training array (should never be eval... might delete stuff) in format needed for dataloader 
        """
        if self.training:
            return self.train_array[index]
        return self.eval_array[index]
    
    def __len__(self):
        """
        Return the length of the training array (once again, might want to delete extra code for eval)
        """
        if self.training:
            return len(self.train_array[:,0])
        return len(self.eval_array[:,0])
    
    def get_eval_data(self):
        """ Return the entire validation dataset """
        return self.eval_array
    
    def get_training_data(self):
        """ Return the entire training dataset """
        return self.train_array

# Testing code to ensure the class works

In [6]:
root_path = "/depot/darkmatter/data/jupyterhub/Physics_Undergrads/Steve/things"

file = root_path + "/all_1.root"

data_o = ProductionModeDataset(file, correlation_cut = 0.8, cut_version=-1)

loaded correlations... shape is (6,)
['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta

In [11]:
data_o.__getitem__(0)

array([3.72715515e-01, 2.58894116e-02, 3.45254717e-02, 1.99258939e-01,
       1.17393606e-01, 9.01361103e-02, 1.81657309e-03, 2.44576740e-01,
       3.05554165e-01, 8.84354442e-01, 6.65447651e-01, 5.72729012e-01,
       2.87516328e-01, 2.91670475e-01, 7.07460533e-02, 1.89763779e-02,
       3.56920362e-01, 2.67904959e-01, 2.19759834e-01, 3.09048836e-01,
       8.92343914e-02, 2.80172142e-01, 3.01306707e-01, 2.76023044e-01,
       6.24736169e-01, 4.80638790e-02, 5.27162548e-01, 1.07177735e-01,
       4.17956806e-01, 1.17960623e-01, 3.54601881e-01, 2.98531575e-04,
       4.81094593e-01, 6.23694164e-02, 2.60965951e-01, 8.37408404e-01,
       2.35444133e-01, 8.18387333e-01, 8.35135937e-01, 8.48587194e-01,
       3.49537976e-01, 1.78700005e-01, 9.57125714e-01, 3.66311318e-01,
       5.82054723e-01, 2.34551295e-01, 6.52294333e-01, 7.65744429e-02,
       2.95814529e-01, 3.85171319e-01, 3.51287262e-01, 2.86013961e-02,
       3.40297469e-02, 7.06269939e-02, 7.25539916e-02, 2.20674442e-02,
      

In [6]:
# data = data_o.get_eval_data()
# data

array([[3.13525153e-01, 9.91798033e-02, 4.54658588e-01, ...,
        0.00000000e+00, 6.25184078e-01, 3.23028000e+05],
       [2.96012653e-01, 9.31698965e-02, 2.77613574e-02, ...,
        2.00000000e+00, 5.97872822e-01, 1.22317000e+05],
       [6.80722665e-01, 2.12740390e-01, 2.79750833e-01, ...,
        2.00000000e+00, 6.40147838e-01, 2.79218000e+05],
       ...,
       [2.89145781e-01, 5.40879933e-01, 1.00308518e-01, ...,
        2.00000000e+00, 6.23233559e-01, 2.87315000e+05],
       [1.30235214e-01, 3.13092280e-01, 3.24186193e-02, ...,
        1.00000000e+00, 6.43382026e-01, 1.01885000e+05],
       [1.59179573e-01, 2.32185964e-01, 1.51536304e-01, ...,
        0.00000000e+00, 6.78623185e-01, 3.15352000e+05]])

In [7]:
# data[:,81]

array([0.62518408, 0.59787282, 0.64014784, ..., 0.62323356, 0.64338203,
       0.67862319])